# 01 - 统计图表基础 (Chart.js)

## 学习目标
- 掌握使用 Chart.js 创建基础统计图表
- 学习 Kotlin 数据生成和 JavaScript 可视化的数据流
- 理解柱状图、折线图、饼图、散点图的应用场景

## 前置知识
- Kotlin 基础语法
- JavaScript 基础
- 数据可视化概念

## 难度: ⭐⭐☆☆☆


In [ ]:
// 🔧 本地调试版本
// 注意：使用本地版本需要先运行 ./gradlew publishToMavenLocal 构建本地版本
// 明确指定使用本地 Maven 仓库
USE {
    repositories {
        mavenLocal()
        mavenCentral()
    }
    dependencies {
        implementation("dev.yidafu.jupyter:jupyter-js:0.8.0")
    }
}

// 配置 Chart.js 库
jsConfig {
    library("chartjs")
}


## 1. 柱状图 - 销售数据分析

让我们从最常用的柱状图开始，展示月度销售数据。


In [ ]:
// 生成销售数据
val salesData = mapOf(
    "1月" to 120000,
    "2月" to 135000,
    "3月" to 142000,
    "4月" to 128000,
    "5月" to 156000,
    "6月" to 168000,
    "7月" to 175000,
    "8月" to 162000,
    "9月" to 148000,
    "10月" to 159000,
    "11月" to 171000,
    "12月" to 185000
)

// 导出数据到 JavaScript
jsExport("salesData", salesData)

println("销售数据已导出，包含 ${salesData.size} 个月的数据")


In [ ]:
%js
import { salesData } from '@jupyter';

// 创建柱状图 - 只负责渲染，不处理数据
const ctx = document.createElement('canvas');
ctx.width = 800;
ctx.height = 400;
document.body.appendChild(ctx);

const chart = new Chart(ctx, {
    type: 'bar',
    data: {
        labels: Object.keys(salesData),
        datasets: [{
            label: '月度销售额 (元)',
            data: Object.values(salesData),
            backgroundColor: 'rgba(54, 162, 235, 0.6)',
            borderColor: 'rgba(54, 162, 235, 1)',
            borderWidth: 1
        }]
    },
    options: {
        responsive: true,
        plugins: {
            title: {
                display: true,
                text: '2024年月度销售数据'
            },
            legend: {
                display: true
            }
        },
        scales: {
            y: {
                beginAtZero: true,
                ticks: {
                    callback: function(value) {
                        return '¥' + value.toLocaleString();
                    }
                }
            }
        }
    }
});

console.log('柱状图已创建，显示月度销售趋势');


## 2. 折线图 - 温度变化趋势

接下来我们创建一个折线图来展示温度变化趋势。所有数据处理在Kotlin中完成。


In [ ]:
// 生成温度数据并预处理
val temperatureData = listOf(
    mapOf("hour" to "00:00", "temperature" to 18.5),
    mapOf("hour" to "02:00", "temperature" to 17.2),
    mapOf("hour" to "04:00", "temperature" to 16.8),
    mapOf("hour" to "06:00", "temperature" to 17.5),
    mapOf("hour" to "08:00", "temperature" to 19.2),
    mapOf("hour" to "10:00", "temperature" to 22.1),
    mapOf("hour" to "12:00", "temperature" to 25.3),
    mapOf("hour" to "14:00", "temperature" to 27.8),
    mapOf("hour" to "16:00", "temperature" to 26.5),
    mapOf("hour" to "18:00", "temperature" to 23.9),
    mapOf("hour" to "20:00", "temperature" to 21.2),
    mapOf("hour" to "22:00", "temperature" to 19.8)
)

// 在Kotlin中处理数据，提取标签和数值
val timeLabels = temperatureData.map { it["hour"] as String }
val temperatureValues = temperatureData.map { it["temperature"] as Double }

// 计算统计信息
val maxTemp = temperatureValues.maxOrNull() ?: 0.0
val minTemp = temperatureValues.minOrNull() ?: 0.0
val avgTemp = temperatureValues.average()

// 导出处理好的数据
jsExport("timeLabels", timeLabels)
jsExport("temperatureValues", temperatureValues)
jsExport("maxTemp", maxTemp)
jsExport("minTemp", minTemp)
jsExport("avgTemp", avgTemp)

println("温度数据已处理并导出:")
println("最高温度: ${maxTemp}°C")
println("最低温度: ${minTemp}°C")
println("平均温度: ${String.format("%.1f", avgTemp)}°C")


In [ ]:
%js
import { timeLabels, temperatureValues, maxTemp, minTemp, avgTemp } from '@jupyter';

// 创建折线图 - 只负责渲染预处理的数据
const lineCtx = document.createElement('canvas');
lineCtx.width = 800;
lineCtx.height = 400;
document.body.appendChild(lineCtx);

const lineChart = new Chart(lineCtx, {
    type: 'line',
    data: {
        labels: timeLabels,
        datasets: [{
            label: '温度 (°C)',
            data: temperatureValues,
            borderColor: 'rgba(255, 99, 132, 1)',
            backgroundColor: 'rgba(255, 99, 132, 0.2)',
            borderWidth: 2,
            fill: true,
            tension: 0.4
        }]
    },
    options: {
        responsive: true,
        plugins: {
            title: {
                display: true,
                text: `24小时温度变化趋势 (最高: ${maxTemp}°C, 最低: ${minTemp}°C, 平均: ${avgTemp.toFixed(1)}°C)`
            }
        },
        scales: {
            y: {
                beginAtZero: false,
                min: Math.floor(minTemp) - 2,
                max: Math.ceil(maxTemp) + 2,
                ticks: {
                    callback: function(value) {
                        return value + '°C';
                    }
                }
            }
        }
    }
});

console.log('折线图已创建，显示温度变化趋势');


## 3. 饼图 - 市场份额分析

现在让我们创建一个饼图来展示市场份额分布。所有数据计算在Kotlin中完成。


In [ ]:
// 生成市场份额数据并计算百分比
val marketShareData = mapOf(
    "苹果" to 28.5,
    "三星" to 22.1,
    "华为" to 15.8,
    "小米" to 12.3,
    "OPPO" to 8.7,
    "vivo" to 6.9,
    "其他" to 5.7
)

// 在Kotlin中处理数据
val brandNames = marketShareData.keys.toList()
val marketValues = marketShareData.values.toList()
val totalMarket = marketValues.sum()
val topBrand = marketShareData.maxByOrNull { it.value }

// 计算排名
val sortedBrands = marketShareData.toList().sortedByDescending { it.second }

// 导出处理好的数据
jsExport("brandNames", brandNames)
jsExport("marketValues", marketValues)
jsExport("totalMarket", totalMarket)
jsExport("topBrand", topBrand?.key)
jsExport("topBrandShare", topBrand?.value)

println("市场份额数据已处理:")
println("总市场份额: ${totalMarket}%")
println("领先品牌: ${topBrand?.key} (${topBrand?.value}%)")
println("品牌排名: ${sortedBrands.take(3).joinToString { "${it.first}: ${it.second}%" }}")


In [ ]:
%js
import { brandNames, marketValues, totalMarket, topBrand, topBrandShare } from '@jupyter';

// 创建饼图 - 只负责渲染预处理的数据
const pieCtx = document.createElement('canvas');
pieCtx.width = 600;
pieCtx.height = 400;
document.body.appendChild(pieCtx);

const pieChart = new Chart(pieCtx, {
    type: 'pie',
    data: {
        labels: brandNames,
        datasets: [{
            data: marketValues,
            backgroundColor: [
                '#FF6384',
                '#36A2EB',
                '#FFCE56',
                '#4BC0C0',
                '#9966FF',
                '#FF9F40',
                '#FF6384'
            ],
            borderWidth: 2,
            borderColor: '#fff'
        }]
    },
    options: {
        responsive: true,
        plugins: {
            title: {
                display: true,
                text: `智能手机市场份额分布 (总计: ${totalMarket}%)`
            },
            legend: {
                position: 'right'
            },
            tooltip: {
                callbacks: {
                    label: function(context) {
                        return context.label + ': ' + context.parsed + '%';
                    }
                }
            }
        }
    }
});

console.log(`饼图已创建，显示市场份额分布。领先品牌: ${topBrand} (${topBrandShare}%)`);


## 4. 散点图 - 身高体重关系分析

最后，我们创建一个散点图来分析身高和体重的关系。所有统计分析在Kotlin中完成。


In [ ]:
// 生成身高体重数据
val heightWeightData = listOf(
    mapOf("height" to 160, "weight" to 55),
    mapOf("height" to 165, "weight" to 60),
    mapOf("height" to 170, "weight" to 65),
    mapOf("height" to 175, "weight" to 70),
    mapOf("height" to 180, "weight" to 75),
    mapOf("height" to 185, "weight" to 80),
    mapOf("height" to 162, "weight" to 58),
    mapOf("height" to 168, "weight" to 62),
    mapOf("height" to 172, "weight" to 68),
    mapOf("height" to 178, "weight" to 72),
    mapOf("height" to 183, "weight" to 78),
    mapOf("height" to 188, "weight" to 85)
)

// 在Kotlin中进行统计分析
val heights = heightWeightData.map { it["height"] as Int }
val weights = heightWeightData.map { it["weight"] as Int }

// 计算基本统计量
val avgHeight = heights.average()
val avgWeight = weights.average()
val minHeight = heights.minOrNull() ?: 0
val maxHeight = heights.maxOrNull() ?: 0
val minWeight = weights.minOrNull() ?: 0
val maxWeight = weights.maxOrNull() ?: 0

// 计算相关系数 (简化版)
val n = heights.size
val sumH = heights.sum()
val sumW = weights.sum()
val sumHW = heights.zip(weights).sumOf { it.first * it.second }
val sumH2 = heights.sumOf { it * it }
val sumW2 = weights.sumOf { it * it }

val correlation = (n * sumHW - sumH * sumW) / 
    kotlin.math.sqrt((n * sumH2 - sumH * sumH) * (n * sumW2 - sumW * sumW).toDouble())

// 准备散点图数据
val scatterData = heights.zip(weights).map { mapOf("x" to it.first, "y" to it.second) }

// 导出处理好的数据
jsExport("scatterData", scatterData)
jsExport("avgHeight", avgHeight)
jsExport("avgWeight", avgWeight)
jsExport("correlation", correlation)

println("身高体重数据分析完成:")
println("平均身高: ${String.format("%.1f", avgHeight)} cm")
println("平均体重: ${String.format("%.1f", avgWeight)} kg")
println("身高范围: ${minHeight} - ${maxHeight} cm")
println("体重范围: ${minWeight} - ${maxWeight} kg")
println("相关系数: ${String.format("%.3f", correlation)}")


In [ ]:
%js
import { scatterData, avgHeight, avgWeight, correlation } from '@jupyter';

// 创建散点图 - 只负责渲染预处理的数据
const scatterCtx = document.createElement('canvas');
scatterCtx.width = 800;
scatterCtx.height = 400;
document.body.appendChild(scatterCtx);

const scatterChart = new Chart(scatterCtx, {
    type: 'scatter',
    data: {
        datasets: [{
            label: '身高体重关系',
            data: scatterData,
            backgroundColor: 'rgba(75, 192, 192, 0.6)',
            borderColor: 'rgba(75, 192, 192, 1)',
            pointRadius: 8,
            pointHoverRadius: 10
        }]
    },
    options: {
        responsive: true,
        plugins: {
            title: {
                display: true,
                text: `身高与体重关系散点图 (相关系数: ${correlation.toFixed(3)})`
            }
        },
        scales: {
            x: {
                title: {
                    display: true,
                    text: '身高 (cm)'
                },
                min: 155,
                max: 195
            },
            y: {
                title: {
                    display: true,
                    text: '体重 (kg)'
                },
                min: 50,
                max: 90
            }
        }
    }
});

console.log(`散点图已创建，显示身高体重关系。平均身高: ${avgHeight.toFixed(1)}cm, 平均体重: ${avgWeight.toFixed(1)}kg`);


## 总结

在这个示例中，我们学习了如何使用 Chart.js 创建四种基础统计图表，并展示了 **Kotlin 数据处理 + JavaScript 渲染** 的最佳实践：

### 1. 柱状图 - 销售数据分析
- **Kotlin**: 生成销售数据，计算统计信息
- **JavaScript**: 渲染柱状图，显示月度趋势

### 2. 折线图 - 温度变化趋势  
- **Kotlin**: 处理温度数据，计算最高/最低/平均温度
- **JavaScript**: 渲染折线图，显示温度变化曲线

### 3. 饼图 - 市场份额分析
- **Kotlin**: 计算市场份额，排序品牌，找出领先者
- **JavaScript**: 渲染饼图，显示比例分布

### 4. 散点图 - 身高体重关系
- **Kotlin**: 计算相关系数，统计分析，数据预处理
- **JavaScript**: 渲染散点图，显示变量关系

### 关键学习点：
- **数据分离**: Kotlin 负责所有数据处理和计算，JavaScript 只负责可视化渲染
- **类型安全**: Kotlin 提供类型安全的数据处理
- **性能优化**: 复杂计算在 Kotlin 中完成，避免 JavaScript 性能瓶颈
- **可维护性**: 数据处理逻辑集中在 Kotlin 中，便于测试和维护

### 扩展练习：
1. 尝试添加更多的统计计算（如标准差、中位数等）
2. 实现数据的动态更新和图表刷新
3. 添加数据导出功能（CSV、JSON）
4. 创建组合图表（如柱状图+折线图）


啊哟